In [260]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Quandl

dow_jones_codes = pd.read_csv("C:/Users/JohnSmith2/Downloads/dowjonesA.csv")
dj_df = pd.DataFrame(dow_jones_codes)
dj_df = dj_df.ix[:,['name','free_code']]

def get_rolling_mean(values, window):
    """Return rolling mean of given values, using specified window size."""
    return pd.rolling_mean(values, window=window)
def get_rolling_std(values, window):
    """Return rolling standard deviation of given values, using specified window size"""
    return pd.rolling_std(values, window=window)
def get_bollinger_bands(rm, rstd):
    """Return upper and lower Bollinger Bands."""
    upper_band = rm + rstd * 2
    lower_band = rm - rstd * 2
    return upper_band, lower_band
def compute_daily_returns(df):
    """Compute and return the daily return values."""
    daily_returns = (df / df.shift(1)) - 1
    daily_returns.ix[0,:] = 0 #Pandas leaves the 0th row full of NaNs
    return daily_returns
def plot_data(df, title="Stock prices", xlabel="Date", ylabel="Price"):
    """Plot stock prices with a custom title and meaningful axis labels."""
    ax = df.plot(title=title, fontsize=12)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()

In [213]:
def retrieveDowJonesData(dj_df):
    dowJonesDict = {}
    for name, value in zip(dj_df['name'], dj_df['free_code']):
        each = Quandl.get(value, trim_start="2010-04-15", trim_end="2016-04-15", authtoken="gEC9xAKi4avigPpoQPX1",
                         returns='numpy')
        df = pd.DataFrame(each)
        dowJonesDict[name] = pd.DataFrame(df)
    print len(dowJonesDict)
    
    return dowJonesDict
dowJonesDict = retrieveDowJonesData(dj_df)
print dowJonesDict.keys()

30
['Johnson & Johnson', 'IBM', 'United Technologies', 'Nike', 'General Electric', 'The Home Depot', 'Verizon', "McDonald's", 'American Express', 'Coca-Cola', 'Caterpillar', 'Visa', 'Walt Disney', 'Boeing', 'Intel', 'Pfizer', 'Procter & Gamble', 'Wal-Mart', 'Merck', 'UnitedHealth Group', 'Cisco Systems', '3M', 'Microsoft', 'JPMorgan Chase', 'Apple', 'Travelers', 'DuPont', 'Goldman Sachs', 'Chevron', 'ExxonMobil']


In [167]:
def createAdjVolAndCloseDfs(dowJonesDict):
    i = 0
    for each in dowJonesDict.keys():
        if i == 0:
            df = dowJonesDict[each]
            adj_close_df = df[['Date','Adj. Close']].rename(columns={'Adj. Close': each})
            adj_vol_df = df[['Date','Adj. Volume']].rename(columns={'Adj. Volume': each})
            i = 1
        else:
            df = dowJonesDict[each]
            next_adj_close_df = df[['Date','Adj. Close']].rename(columns={'Adj. Close': each})
            next_adj_vol_df = df[['Date','Adj. Volume']].rename(columns={'Adj. Volume': each})
            adj_close_df = pd.merge(adj_close_df, next_adj_close_df, on='Date')
            adj_vol_df = pd.merge(adj_vol_df, next_adj_vol_df, on='Date')
            
    print adj_close_df.shape
    print adj_vol_df.shape
    
    return adj_close_df, adj_vol_df
adj_close_df, adj_vol_df = createAdjVolAndCloseDfs(dowJonesDict)

(1491, 31)
(1491, 31)


In [261]:
def computeAndPlotBollingerBands(adj_close_df):
    #Compute Bollinger Bands
    # 1. Compute rolling mean
    adj_close_df = adj_close_df.set_index(['Date'])
    i = 0
    for each in adj_close_df:
        rm_SPY = get_rolling_mean(adj_close_df[each], window=20)
        # 2. Compute rolling standard deviation
        rstd_SPY = get_rolling_std(adj_close_df[each], window=20)
        # 3. Compute upper and lower bands
        upper_band, lower_band = get_bollinger_bands(rm_SPY, rstd_SPY)

        if i == 0:
            upper_band_df = pd.DataFrame(upper_band)
            lower_band_df = pd.DataFrame(lower_band)
            i = 1
        else:
            next_upper_df = pd.DataFrame(upper_band)
            next_lower_df = pd.DataFrame(lower_band)
            upper_band_df = pd.merge(upper_band_df, next_upper_df, left_index=True, right_index=True)
            lower_band_df = pd.merge(lower_band_df, next_lower_df, left_index=True, right_index=True)
    
    upper_band_df = upper_band_df[20:]
    lower_band_df = lower_band_df[20:]
    
    print upper_band_df.shape
    print lower_band_df.shape
    
    """
    # Plot raw SPY values, rolling mean and Bollinger Bands
    ax = adj_close_df['IBM'][20:160].plot(title="Bollinger Bands", label='IBM')
    rm_SPY.plot(label='Rolling mean', ax=ax)
    upper_band.plot(label='upper band', ax=ax)
    lower_band.plot(label='lower band', ax=ax)

    # Add axis labels and legend
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend(loc='upper left')
    plt.show()
    """
    
    return upper_band_df, lower_band_df
    
def computeAndPlotDailyReturns(adj_close_df):
    #Compute daily returns
    adj_close_df = adj_close_df.set_index(['Date'])
    adj_close_df = adj_close_df[['IBM','Nike']][0:5]
    print adj_close_df
    daily_returns = compute_daily_returns(adj_close_df)
    plot_data(daily_returns, title="Daily returns", ylabel="Daily returns")

#computeAndPlotDailyReturns(adj_close_df)
upper_band_df, lower_band_df = computeAndPlotBollingerBands(adj_close_df)

(1471, 30)
(1471, 30)
